In [18]:
import boto3
import pandas as pd
import os.path
import matplotlib.pyplot as plt
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [35]:
s3 = boto3.resource('s3')
client = boto3.client("s3")

bucket = "ehrlich-arb-data"
dateid = "20171207"

buck = s3.Bucket(bucket)

s3files = []
for o in buck.objects.filter(Prefix=dateid).all():
    s3files.append(o.key)

if os.path.isdir(dateid) == False:
    os.mkdir(dateid)
print("%s files found in %s" % (len(s3files), bucket))

1 files found in ehrlich-arb-data


In [38]:
for f in s3files:
    if os.path.isfile(f) == False:
        print("Downloading %s ..." % (f), end='')
        client.download_file(bucket, f, f)
        print("done")

In [39]:
files = []
for file in os.listdir(dateid):
    files.append(dateid + "/" + file)
    print(file)

ls.s3.a8db3539-3e52-4850-8283-6b1a48448b4d.2017-12-07T01.40.part0.txt.gz


In [40]:
l = []
d = {'ts': float, 'exchange': str, 'tradingPair': str, 'orderType': str, 'price': float, 'amount': float}
n = ["ts", 'exchange', 'tradingPair', 'orderType', 'price', 'amount']
for (i, f) in enumerate(files):
    df = pd.read_csv(f, dtype = d, header=None, names = n, error_bad_lines = False, na_values = ["Connection aborted"])
    l.append(df)

In [41]:
data = pd.concat(l)

In [42]:
data.dtypes

ts             float64
exchange        object
tradingPair     object
orderType       object
price          float64
amount         float64
dtype: object

In [43]:
data.head()

,ts,exchange,tradingPair,orderType,price,amount
0,1.512611e+12,BITTREX,DASHBTC,BUY,0.000284,154.032365
1,1.512611e+12,BITTREX,DASHBTC,SELL,0.048650,5.205923
2,1.512611e+12,BITTREX,DASHBTC,BUY,0.047201,1.756857
3,1.512611e+12,BITTREX,DASHBTC,SELL,0.000739,62.338000
4,1.512611e+12,BITTREX,DASHBTC,SELL,0.000739,237.618847


In [63]:
data.where(data.tradingPair == "PPCBTC").where(data.orderType == "BUY").price.describe()

count    68248.000000
mean         0.000116
std          0.000075
min          0.000010
25%          0.000050
50%          0.000100
75%          0.000170
max          0.000270
Name: price, dtype: float64

In [78]:
data['dt'] = pd.to_datetime(data['ts']/1000,unit='s')
data.where(data.tradingPair == "PPCBTC").where(data.orderType == "BUY")[['dt', 'price']].plot(figsize=(10, 8))

ValueError: ordinal must be >= 1

ValueError: ordinal must be >= 1